In [1]:
from astropy.table import Table
from dendrocat import RadioSource
from astropy.io import fits
from spectral_cube import SpectralCube
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import sys
import importlib
from astropy.wcs import WCS
from scipy import interpolate
import pandas as pd
import sympy as syp
from astropy import units as u
import radio_beam
from astropy.table import Table
from radio_beam import Beam
from astropy import coordinates
from astropy import wcs
from astropy.nddata.utils import Cutout2D
from dendrocat.aperture import Ellipse
from astropy.io import ascii
import astropy.constants as c

from functools import reduce
from itertools import chain
W51 = '/orange/adamginsburg/w51/TaehwaYoo/'
W51b6 = '/orange/adamginsburg/w51/TaehwaYoo/2015.1.01596.S_W51_B6_LB/continuum_images/'
W51cont='/orange/adamginsburg/w51/TaehwaYoo/b6contfits/'

#---------- high res B6 (before conv) ------------
w51e2_b6_briggs=W51cont+'W51e2_cont_bigbriggs.image.fits'
w51e2_b6_robust0=W51cont+'W51e2_cont_big_robust0.image.fits'
w51e2_b6_uniform=W51cont+'W51e2_cont_biguniform.image.fits'
w51e2_b6_superuniform=W51cont+'W51e2_cont_bigsuperuniform.image.fits'

w51n_b6_briggs = W51cont+'W51n_cont_bigbriggs.image.fits'
w51n_b6_robust0 = W51cont+'w51n_cont_big_robust0.image.fits'
w51n_b6_uniform = W51cont+'W51n_cont_biguniform.image.fits'
w51n_b6_superuniform = W51cont+'W51n_cont_bigsuperuniform.image.fits'
w51n_b6_natural =  W51cont+'W51n_cont_bignatural.image.fits'

#---------- high res B3 (before conv) ------------

W51b3 = '/orange/adamginsburg/w51/TaehwaYoo/2017.1.00293.S_W51_B3_LB/may2021_successful_imaging/'

w51n_b3_tt0 = W51b3+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b3_tt1 = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51n_b3_alpha = W51+'w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51conv = '/orange/adamginsburg/w51/TaehwaYoo/convolved_new/'
w51n_b6_conv = w51conv + 'w51n_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b3_tt0 = W51b3+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51e2_b3_tt1 = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt1.pbcor.fits'
w51e2_b3_alpha = W51+'w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.alpha.pbcor.fits'

w51e2_b6_conv = w51conv + 'w51e2_cont_bigbriggs.image.convB3_briggs.fits'

w51e_b6_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B6_conv.fits'
w51e_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51e_B3_conv.fits'


w51n_b3_almaimf_conv = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/w51n_B3_conv.fits'

w51n_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'

w51n_b6_conv_briggs = w51conv + 'w51n_briggs_B6_conv.fits'
w51n_b6_conv_robust0 = w51conv + 'w51n_robust0_B6_conv.fits'
w51n_b6_conv_superuniform = w51conv + 'w51n_superuniform_B6_conv.fits'

w51e_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'

w51e_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51e_matched.fits'
w51n_matched_catalog = '/home/t.yoo/w51/catalogue/dendrogram/dendro_w51n_matched.fits'

#catalogdir = '/Users/dbahck37/w51data/alma_imf/catalog/getsf-native/'
catalogdir = '/orange/adamginsburg/ALMA_IMF/catalogs/getsf-smoothed/'
cat_w51e = catalogdir + 'W51-E-getsf.cat'
cat_w51n = catalogdir + 'W51-IRS2-getsf.cat'


w51n_b3_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B3_uid___A001_X1296_X18f_continuum_merged_12M_robust0_selfcal4_finaliter.image.tt0.pbcor.fits'
w51n_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'
w51e_b3_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-E_B3_uid___A001_X1296_X10b_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'
w51e_b6_almaimf = '/orange/adamginsburg/w51/TaehwaYoo/w51_alma_imf/W51-E_B6_uid___A001_X1296_X213_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'
w51e_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51e2.spw0thru19.14500.robust0.thr0.15mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
w51n_b6_calibrated_pbcor = '/orange/adamginsburg/w51/TaehwaYoo/w51n.spw0thru19.14500.robust0.thr0.1mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'
#---------- local direc  ------------

"""

w51n_b3_almaimf = '/Users/dbahck37/w51data/alma_imf/W51-IRS2_B3_uid___A001_X1296_X18f_continuum_merged_12M_robust0_selfcal4_finaliter.image.tt0.pbcor.fits'
w51n_b6_almaimf = '/Users/dbahck37/w51data/alma_imf/W51-IRS2_B6_uid___A001_X1296_X187_continuum_merged_12M_robust0_selfcal9_finaliter.image.tt0.pbcor.fits'



w51e_b3_almaimf = '/Users/dbahck37/w51data/alma_imf/W51-E_B3_uid___A001_X1296_X10b_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'
w51e_b6_almaimf = '/Users/dbahck37/w51data/alma_imf/W51-E_B6_uid___A001_X1296_X213_continuum_merged_12M_robust0_selfcal7_finaliter.image.tt0.pbcor.fits'


w51n_b6_conv_briggs = '/Users/dbahck37/w51data/convolved_new/w51n_briggs_B6_conv.fits'
w51n_b3_tt0 = '/Users/dbahck37/w51data/w51n.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'

w51e_b3_tt0 = '/Users/dbahck37/W51data/w51e2.spw0thru19.14500.robust0.thr0.075mJy.mfs.I.startmod.selfcal7.image.tt0.pbcor.fits'

w51e_b6_conv_briggs ='/Users/dbahck37/W51data/convolved_new/w51e_briggs_B6_conv.fits'
"""

w51e_almaimf_coretemp = '/home/t.yoo/W51-E_core_temperature_smooth_catalog.dat'
w51n_almaimf_coretemp = '/home/t.yoo/W51-IRS2_core_temperature_smooth_catalog.dat'


class catdata():
    def __init__(self,catfile,band='B3'):
        catdata_w51e = ascii.read(catfile,data_start=0,format='commented_header', header_start=120,  comment="!")
        self.sky_ra = catdata_w51e['WCS_ACOOR']
        self.sky_dec = catdata_w51e['WCS_DCOOR']
        self.pix_x = catdata_w51e['XCO_P']
        self.pix_y = catdata_w51e['YCO_P']
        
        if band=='B3':
            bandind = 3
        else:
            bandind = 2

        self.afwhm = catdata_w51e['AFWHM0%d'%bandind] #arcsec
        self.bfwhm = catdata_w51e['BFWHM0%d'%bandind]

        self.asize = catdata_w51e['ASIZE0%d'%bandind]
        self.bsize = catdata_w51e['BSIZE0%d'%bandind]

        self.theta = catdata_w51e['THETA0%d'%bandind]
        
        self.flux = catdata_w51e['FXT_BST0%d'%bandind]
        self.fluxerr = catdata_w51e['FXT_ERR0%d'%bandind]
        self.flux_g = catdata_w51e['FXT_ALT0%d'%bandind]

        self.fooa = catdata_w51e['FOOA0%d'%bandind]
        self.foob = catdata_w51e['FOOB0%d'%bandind]
        
        self.int_peak = catdata_w51e['FXP_BST0%d'%bandind]
        self.interr_peak = catdata_w51e['FXP_ERR0%d'%bandind]
        
        self.size = (catdata_w51e['AFWHM0%d'%bandind]*catdata_w51e['BFWHM0%d'%bandind]**2)**(1/3)


        self.no = catdata_w51e['NO']

        self.rec_ind_b3 = np.where((np.abs(catdata_w51e['GOODM03'])>1)&
                           (np.abs(catdata_w51e['SIGNM03'])>1)&
                           (catdata_w51e['FXP_BST03']/catdata_w51e['FXP_ERR03']>2)&
                           (catdata_w51e['FXT_BST03']/catdata_w51e['FXT_ERR03']>2)&
                           (catdata_w51e['AFWHM03']/catdata_w51e['BFWHM03']<2)&
                           (catdata_w51e['FOOA03']/catdata_w51e['AFWHM03']>1.15))[0]

        self.rec_ind_b6 = np.where((np.abs(catdata_w51e['GOODM02'])>1)&
                           (np.abs(catdata_w51e['SIGNM02'])>1)&
                           (catdata_w51e['FXP_BST02']/catdata_w51e['FXP_ERR02']>2)&
                           (catdata_w51e['FXT_BST02']/catdata_w51e['FXT_ERR02']>2)&
                           (catdata_w51e['AFWHM02']/catdata_w51e['BFWHM02']<2)&
                           (catdata_w51e['FOOA02']/catdata_w51e['AFWHM02']>1.15))[0]
        self.rec_ind = np.where((np.abs(catdata_w51e['GOODM03'])>1)&
                           (np.abs(catdata_w51e['SIGNM03'])>1)&
                           (catdata_w51e['FXP_BST03']/catdata_w51e['FXP_ERR03']>2)&
                           (catdata_w51e['FXT_BST03']/catdata_w51e['FXT_ERR03']>2)&
                           (catdata_w51e['AFWHM03']/catdata_w51e['BFWHM03']<2)&
                           (catdata_w51e['FOOA03']/catdata_w51e['AFWHM03']>1.15) &
                           (np.abs(catdata_w51e['GOODM02'])>1) &
                           (np.abs(catdata_w51e['SIGNM02'])>1)&
                           (catdata_w51e['FXP_BST02']/catdata_w51e['FXP_ERR02']>2)&
                           (catdata_w51e['FXT_BST02']/catdata_w51e['FXT_ERR02']>2)&
                           (catdata_w51e['AFWHM02']/catdata_w51e['BFWHM02']<2)&
                           (catdata_w51e['FOOA02']/catdata_w51e['AFWHM02']>1.15))[0]
        
        rescaled_3mmflux = catdata_w51e['FXT_BST03'] * catdata_w51e['AFWHM02']/catdata_w51e['AFWHM03']
        gamma = np.log10(catdata_w51e['FXT_BST02'] / rescaled_3mmflux)/np.log10(228.918/101.426)
        self.woffind = np.where(gamma>2)[0]
        self.wffind = np.where(gamma<2)[0]

        
    def get_temperature(self,tempfile,threshold=1e-4):
        tab = ascii.read(tempfile, data_start=4)
        ra = tab['col2']
        dec = tab['col3']
        temp = tab['col4']

        temparr = []
        for i in range(len(ra)):
            dist = np.sqrt((ra[i]-self.sky_ra)**2+(dec[i]-self.sky_dec)**2)
            mindist = np.min(dist)
            if mindist<threshold:
                index = np.argmin(dist)
                temparr.append(temp[index])
            else:
                temparr.append(np.nan)
        self.temp = np.array(temparr)*u.K
        return np.array(temparr)
    
    def get_coremass(self,freq, kappa, dist):
       
        mass = - 1 / kappa * dist**2 * self.flux  / self.int_peak * np.log(1 - (self.int_peak *u.Jy  / BB(freq,self.temp)).to(u.cm/u.cm).value)
        self.mass = mass.to(u.Msun)
        return mass.to(u.Msun)
    
    def get_coremasserr(self,freq, kappa, dist,):
        
        masserr = np.abs(self.mass * np.sqrt((self.fluxerr/self.flux)**2+(self.interr_peak/self.int_peak)**2))
        return masserr.to(u.Msun)   
    
    
        
        
    

                
            
def BB(freq, temp):
    B_nu = (2 * freq**3 *c.h / (c.c**2) * 1 / (np.e**(c.h*freq/(c.k_B*temp))-1))
    return B_nu        

def kappa(nu, nu0=271.1*u.GHz, kappa0=0.0114*u.cm**2*u.g**-1, beta=1.75):
    """
    Compute the opacity $\kappa$ given a reference frequency (or wavelength)
    and a power law governing the opacity as a fuction of frequency:
    $$ \kappa = \kappa_0 \left(\\frac{\\nu}{\\nu_0}\\right)^{\\beta} $$
    The default kappa=0.0114 at 271.1 GHz comes from extrapolating the
    Ossenkopf & Henning 1994 opacities for the thin-ice-mantle, 10^6 year model
    anchored at 1.0 mm with an assumed beta of 1.75.
    Parameters
    ----------
    nu: astropy.Quantity [u.spectral() equivalent]
        The frequency at which to evaluate kappa
    nu0: astropy.Quantity [u.spectral() equivalent]
        The reference frequency at which $\kappa$ is defined
    kappa0: astropy.Quantity [cm^2/g]
        The dust opacity per gram of H2 along the line of sight.  Because of
        the H2 conversion, this factor implicitly includes a dust to gas ratio
        (usually assumed 100)
    beta: float
        The power-law index governing kappa as a function of nu
    """
    return (kappa0*(nu.to(u.GHz,u.spectral())/nu0.to(u.GHz,u.spectral()))**(beta)).to(u.cm**2/u.g)        

     

def get_BEmass(major, minor, temp ):
    rad = (major * minor**2)**(1/3)
    sigma_th = np.sqrt(c.k_B*temp*u.K/(2.4*c.m_p))
    return (2.4*rad*u.au*sigma_th**2/c.G).to(u.Msun)

def get_Jeansmass(temp, mass, major, minor):
    rad = (major * minor**2)**(1/3)
    rho = mass*(u.Msun) / (4/3*np.pi * (rad*u.au)**3)
    c_s = np.sqrt(c.k_B*temp*u.K/2.8/c.m_p)
    jeans_mass = np.pi**(5/2)/6/c.G**(3/2) * c_s.to(u.cm/u.s)**3 * rho.to(u.g/u.cm**3)**(-1/2)
    return jeans_mass.to(u.Msun)
def count_points_in_ellipse(xcen,ycen,major,minor,pa,xpos,ypos):
    # caclulate foci positions 
    # must be full major, minor not semi major, minor
    fodx = np.cos(pa)*np.sqrt((major/2)**2-(minor/2)**2)
    fody = np.sin(pa)*np.sqrt((major/2)**2-(minor/2)**2)
    
    fox1 = xcen + fodx
    foy1 = ycen + fody
    
    fox2 = xcen - fodx
    foy2 = ycen - fody
    
    dist1 = np.sqrt((xpos-fox1)**2+(ypos-foy1)**2)
    dist2 = np.sqrt((xpos-fox2)**2+(ypos-foy2)**2)
    
    isinside = np.where(dist1+dist2<major)[0]
    
    return len(isinside), list(isinside)

class multiplicity():
    def __init__(self, hdrhr,core_catdatadir,xysource,isb3=True,beam=None,use_criteria='b6',issky=True, label=None,
                 aperture='fwhm+beam',limitarea=None,  maxrad=1e3):
    
        catdata = ascii.read(core_catdatadir,data_start=0,format='commented_header', header_start=120,  comment="!")
        sky_ra = catdata['WCS_ACOOR']
        sky_dec = catdata['WCS_DCOOR']
        no = catdata['NO']
        if isb3:
            bandlabel = '03'
        else:
            bandlabel = '02'
        fooa = catdata['FOOA%s'%bandlabel]
        foob = catdata['FOOB%s'%bandlabel]
        theta = catdata['THETA%s'%bandlabel]
        afwhm = catdata['AFWHM%s'%bandlabel]
        bfwhm = catdata['BFWHM%s'%bandlabel]

        ncat = len(no)
        
        coreidx = np.arange(ncat)

        if use_criteria == 'b6':
            rec_ind = np.where((np.abs(catdata['GOODM02'])>1)&
                       (np.abs(catdata['SIGNM02'])>1)&
                       (catdata['FXP_BST02']/catdata['FXP_ERR02']>2)&
                       (catdata['FXT_BST02']/catdata['FXT_ERR02']>2)&
                       (catdata['AFWHM02']/catdata['BFWHM02']<2)&
                       (catdata['FOOA02']/catdata['AFWHM02']>1.15))[0]
        elif use_criteria == 'b3':
            rec_ind = np.where((np.abs(catdata['GOODM03'])>1)&
                       (np.abs(catdata['SIGNM03'])>1)&
                       (catdata['FXP_BST03']/catdata['FXP_ERR03']>2)&
                       (catdata['FXT_BST03']/catdata['FXT_ERR03']>2)&
                       (catdata['AFWHM03']/catdata['BFWHM03']<2)&
                       (catdata['FOOA03']/catdata['AFWHM03']>1.15))[0]
        elif use_criteria == 'both': 
            rec_ind = np.where((np.abs(catdata['GOODM03'])>1)&
                               (np.abs(catdata['SIGNM03'])>1)&
                               (catdata['FXP_BST03']/catdata['FXP_ERR03']>2)&
                               (catdata['FXT_BST03']/catdata['FXT_ERR03']>2)&
                               (catdata['AFWHM03']/catdata['BFWHM03']<2)&
                               (catdata['FOOA03']/catdata['AFWHM03']>1.15) &
                               (np.abs(catdata['GOODM02'])>1) &
                               (np.abs(catdata['SIGNM02'])>1)&
                               (catdata['FXP_BST02']/catdata['FXP_ERR02']>2)&
                               (catdata['FXT_BST02']/catdata['FXT_ERR02']>2)&
                               (catdata['AFWHM02']/catdata['BFWHM02']<2)&
                               (catdata['FOOA02']/catdata['AFWHM02']>1.15))[0]

        else:
            rec_ind = np.arange(ncat)

        if limitarea is not None:
            posind = np.where((sky_ra>limitarea[0])&(sky_ra<limitarea[1])&
                              (sky_dec>limitarea[2])&(sky_dec<limitarea[3]))[0]
        else:
            posind = np.arange(ncat)
        if label is None:
            label=np.arange(ncat)

        totalind = reduce(np.intersect1d, (rec_ind,  posind, label))  

        sky_ra = sky_ra[totalind]
        sky_dec = sky_dec[totalind]
        fooa = fooa[totalind]
        foob = foob[totalind]
        theta = theta[totalind]
        no = no[totalind]
        afwhm =afwhm[totalind]
        bfwhm = bfwhm[totalind]
        coreidx = coreidx[totalind]

        skypos = np.vstack((sky_ra,sky_dec)).T


        wcsNB = WCS(hdrhr,naxis=2)
        scaleNB = wcsNB.proj_plane_pixel_scales()[0]
        beamNB = Beam.from_fits_header(hdrhr)

        xypos_core = wcsNB.wcs_world2pix(skypos,0)
        xpos_core = xypos_core[:,0]
        ypos_core = xypos_core[:,1]

        if issky:
            xysource_pix = wcsNB.wcs_world2pix(xysource,0)
        else:
            xysource_pix = xysource
        ysonumarr=[] ; coreindarr=[]; ysoindarr=[]
        #print(xysource)
        for i in range(len(xypos_core)):
            if aperture is 'footprint':
                major = fooa[i]/3600/scaleNB.value
                minor = foob[i]/3600/scaleNB.value
                pa = theta[i]+90
            elif aperture is 'fwhm+beam':
                meanbeamsize = (beam.major.value+beam.minor.value)/4
                major = (afwhm[i]/3600+meanbeamsize)/scaleNB.value
                minor = (bfwhm[i]/3600+meanbeamsize)/scaleNB.value
                pa = theta[i]+90
            elif aperture is 'beam':
                major = beam.major.value/scaleNB.value
                minor = beam.minor.value/scaleNB.value
                pa = beam.pa.value+90
            if major > maxrad:
                continue

            numsource, indsource = count_points_in_ellipse(xpos_core[i],ypos_core[i],major,minor,pa,xysource_pix[:,0],xysource_pix[:,1])

            ysonumarr.append(numsource)
            coreindarr.append(coreidx[i])
            ysoindarr.append(indsource)
            
      
        ysoindarr_flat = []
        for arr in ysoindarr:
            ysoindarr_flat.extend(arr)
       
        print('cores with/without YSOs ', len(np.where(np.array(ysonumarr)!=0)[0]), len(xpos_core)-len(np.where(np.array(ysonumarr)!=0)[0]))
        print('YSOs associated with cores',len(np.unique(ysoindarr_flat)),' out of ',len(xysource_pix[:,0]))
              
        self.ysonumarr = ysonumarr
        self.coreindarr = coreindarr
        self.ysoindarr = ysoindarr
        
   
    


def exist_inside_FOV(xcore_sky, ycore_sky, yso_image, hrhdr):
    isfinite = np.where(np.isfinite(yso_image))
    xfinite1 = np.min(isfinite[0])
    xfinite2 = np.max(isfinite[0])
    yfinite1 = np.min(isfinite[1])
    yfinite2 = np.max(isfinite[1])
    
    xcen = (xfinite1 + xfinite2)/2
    ycen = (yfinite1 + yfinite2)/2

    wcs_hr = WCS(hrhdr,naxis=2)
    xypos = wcs_hr.wcs_world2pix(np.vstack((xcore_sky,ycore_sky)).T,0)
    
    rad = np.abs(xfinite2 - xcen)
    
    
    dist = np.sqrt((xypos[:,0]- xcen)**2+(xypos[:,1]-ycen)**2)
    isinside = np.where(dist<rad)[0]
    
    return isinside
     

<>:369: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:373: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:378: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:369: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:373: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:378: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [2]:
freqb3=92982346121.91989*u.Hz
freqb6=226691598706.70853*u.Hz
kappa_b3 = kappa(freqb3)
kappa_b6 = kappa(freqb6)
catdata_w51e_b3 = catdata(cat_w51e)
sky_ra_w51e_b3 = catdata_w51e_b3.sky_ra
sky_dec_w51e_b3 = catdata_w51e_b3.sky_dec
flux_w51e_b3 = catdata_w51e_b3.flux
fluxerr_w51e_b3 = catdata_w51e_b3.fluxerr
size_w51e_b3 = catdata_w51e_b3.size * 5410 #arcsec * pc -> au

catdata_w51e_b6 = catdata(cat_w51e, band='B6')
sky_ra_w51e_b6 = catdata_w51e_b6.sky_ra
sky_dec_w51e_b6 = catdata_w51e_b6.sky_dec
flux_w51e_b6 = catdata_w51e_b6.flux
fluxerr_w51e_b6 = catdata_w51e_b6.fluxerr
size_w51e_b6 = catdata_w51e_b6.size * 5410 #arcsec * pc -> au


catdata_w51n_b3 = catdata(cat_w51n)
sky_ra_w51n_b3 = catdata_w51n_b3.sky_ra
sky_dec_w51n_b3 = catdata_w51n_b3.sky_dec
flux_w51n_b3 = catdata_w51n_b3.flux
fluxerr_w51n_b3 = catdata_w51n_b3.fluxerr
size_w51n_b3 = catdata_w51n_b3.size * 5100 #arcsec * pc -> au


catdata_w51n_b6 = catdata(cat_w51n, band='B6')
sky_ra_w51n_b6 = catdata_w51n_b6.sky_ra
sky_dec_w51n_b6 = catdata_w51n_b6.sky_dec
flux_w51n_b6 = catdata_w51n_b6.flux
fluxerr_w51n_b6 = catdata_w51n_b6.fluxerr
size_w51n_b6 = catdata_w51n_b6.size * 5100 #arcsec * pc -> au

coretemp_w51e = catdata_w51e_b6.get_temperature(w51e_almaimf_coretemp)
coretemp_w51n = catdata_w51n_b6.get_temperature(w51n_almaimf_coretemp)
coremass_w51e = catdata_w51e_b6.get_coremass(freqb6, kappa_b6, 5.41*u.kpc)
coremass_w51n = catdata_w51n_b6.get_coremass(freqb6, kappa_b6, 5.1*u.kpc)
coremasserr_w51e = catdata_w51e_b6.get_coremasserr(freqb6, kappa_b6, 5.41*u.kpc)
coremasserr_w51n = catdata_w51n_b6.get_coremasserr(freqb6, kappa_b6, 5.1*u.kpc)


fitsdata_b3 = fits.open(w51e_b3_tt0)
image_b3_w51e = fitsdata_b3[0].data
if len(image_b3_w51e.shape)>2:
    image_b3_w51e = fitsdata_b3[0].data[0][0]
hdrB3_w51e_hr = fits.getheader(w51e_b3_tt0)  
    
fitsdata_b3 = fits.open(w51n_b3_tt0)
image_b3_w51n = fitsdata_b3[0].data
if len(image_b3_w51n.shape)>2:
    image_b3_w51n = fitsdata_b3[0].data[0][0]
hdrB3_w51n_hr = fits.getheader(w51n_b3_tt0)  

fitsdata_b6 = fits.open(w51e_b6_calibrated_pbcor)
image_b6_w51e = fitsdata_b6[0].data
if len(image_b6_w51e.shape)>2:
    image_b6_w51e = fitsdata_b6[0].data[0][0]
hdrB6_w51e_hr = fits.getheader(w51e_b6_calibrated_pbcor)  
    
fitsdata_b6 = fits.open(w51n_b6_calibrated_pbcor)
image_b6_w51n = fitsdata_b6[0].data
if len(image_b6_w51n.shape)>2:
    image_b6_w51n = fitsdata_b6[0].data[0][0]
hdrB6_w51n_hr = fits.getheader(w51n_b6_calibrated_pbcor)  

hdrB6_w51e_almaimf = fits.getheader(w51e_b6_almaimf)  
wcsB6_w51e_almaimf = WCS(hdrB6_w51e_almaimf,naxis=2)
beamb6_w51e_almaimf = Beam.from_fits_header(hdrB6_w51e_almaimf)  

hdrB3_w51e_almaimf = fits.getheader(w51e_b3_almaimf)  
wcsB3_w51e_almaimf = WCS(hdrB3_w51e_almaimf,naxis=2)
beamb3_w51e_almaimf = Beam.from_fits_header(hdrB3_w51e_almaimf) 

hdrB6_w51n_almaimf = fits.getheader(w51n_b6_almaimf)  
wcsB6_w51n_almaimf = WCS(hdrB6_w51n_almaimf,naxis=2)
beamb6_w51n_almaimf = Beam.from_fits_header(hdrB6_w51n_almaimf)  

hdrB3_w51n_almaimf = fits.getheader(w51n_b3_almaimf)  
wcsB3_w51n_almaimf = WCS(hdrB3_w51n_almaimf,naxis=2)
beamb3_w51n_almaimf = Beam.from_fits_header(hdrB3_w51n_almaimf)  

catalog_w51e_yso = Table.read(w51e_matched_catalog ,format='fits')
w51e_xsky_b6 = catalog_w51e_yso['b6_xsky'] ; w51e_ysky_b6 = catalog_w51e_yso['b6_ysky']
w51e_xsky_b3 = catalog_w51e_yso['b3_xsky'] ; w51e_ysky_b3 = catalog_w51e_yso['b3_ysky']

detect_w51e_b6 = np.where(w51e_xsky_b6>0)[0]
detect_w51e_b3 = np.where(w51e_xsky_b3>0)[0]


catalog_w51n_yso = Table.read(w51n_matched_catalog ,format='fits')
w51n_xsky_b6 = catalog_w51n_yso['b6_xsky'] ; w51n_ysky_b6 = catalog_w51n_yso['b6_ysky']
w51n_xsky_b3 = catalog_w51n_yso['b3_xsky'] ; w51n_ysky_b3 = catalog_w51n_yso['b3_ysky']

xsky_w51e = []
for i,x in enumerate(w51e_xsky_b6):
    if x > 0:
        xsky_w51e.append(x)
    else:
        xsky_w51e.append(w51e_xsky_b3[i])
ysky_w51e = []
for i,x in enumerate(w51e_ysky_b6):
    if x > 0:
        ysky_w51e.append(x)
    else:
        ysky_w51e.append(w51e_ysky_b3[i])

xsky_w51n = []
for i,x in enumerate(w51n_xsky_b6):
    if x > 0:
        xsky_w51n.append(x)
    else:
        xsky_w51n.append(w51n_xsky_b3[i])
ysky_w51n = []
for i,x in enumerate(w51n_ysky_b6):
    if x > 0:
        ysky_w51n.append(x)
    else:
        ysky_w51n.append(w51n_ysky_b3[i])

isinside_w51e_b3 = exist_inside_FOV(sky_ra_w51e_b6, sky_dec_w51e_b6, image_b3_w51e, hdrB3_w51e_hr)
isinside_w51e_b6 = exist_inside_FOV(sky_ra_w51e_b6, sky_dec_w51e_b6, image_b6_w51e, hdrB6_w51e_hr)
isinside_w51n_b3 = exist_inside_FOV(sky_ra_w51n_b6, sky_dec_w51n_b6, image_b3_w51n, hdrB3_w51n_hr)
isinside_w51n_b6 = exist_inside_FOV(sky_ra_w51n_b6, sky_dec_w51n_b6, image_b6_w51n, hdrB6_w51n_hr)

isinside_w51e_b3_woff = np.intersect1d(isinside_w51e_b3,  catdata_w51e_b3.woffind)
isinside_w51n_b3_woff = np.intersect1d(isinside_w51n_b3,  catdata_w51n_b3.woffind)
isinside_w51e_b6_woff = np.intersect1d(isinside_w51e_b6,  catdata_w51e_b6.woffind)
isinside_w51n_b6_woff = np.intersect1d(isinside_w51n_b6,  catdata_w51n_b6.woffind)

isinside_w51e_b3_wff = np.intersect1d(isinside_w51e_b3,  catdata_w51e_b3.wffind)
isinside_w51n_b3_wff = np.intersect1d(isinside_w51n_b3,  catdata_w51n_b3.wffind)
isinside_w51e_b6_wff = np.intersect1d(isinside_w51e_b6,  catdata_w51e_b6.wffind)
isinside_w51n_b6_wff = np.intersect1d(isinside_w51n_b6,  catdata_w51n_b6.wffind)


xysource_w51e= np.vstack((xsky_w51e, ysky_w51e)).T
xysource_w51n= np.vstack((xsky_w51n, ysky_w51n)).T

hdrhr_w51n_b3= fits.getheader(w51n_b3_tt0) 
hdrhr_w51e_b3= fits.getheader(w51e_b3_tt0) 



multiplicity_w51e = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='b6',
                                 label=isinside_w51e_b3)
multiplicity_w51n = multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam = beamb6_w51n_almaimf,
                                 use_criteria='b6',
                                label=isinside_w51n_b3)

multiplicity_w51e_woff = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='b6',
                                 label=isinside_w51e_b3_woff)
multiplicity_w51n_woff = multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam = beamb6_w51n_almaimf,
                                 use_criteria='b6',
                                label=isinside_w51n_b3_woff)

multiplicity_w51e_wff = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='b6',
                                 label=isinside_w51e_b3_wff)
multiplicity_w51n_wff = multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam = beamb6_w51n_almaimf,
                                 use_criteria='b6',
                                label=isinside_w51n_b3_wff)

multiplicity_w51e_b3 = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='both',
                                   label=isinside_w51e_b3)
multiplicity_w51n_b3 =  multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam= beamb6_w51n_almaimf, 
                                 use_criteria='both',
                                    label=isinside_w51n_b3)

multiplicity_w51e_woff_b3 = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='both',
                                 label=isinside_w51e_b3_woff)
multiplicity_w51n_woff_b3 = multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam = beamb6_w51n_almaimf,
                                 use_criteria='both',
                                label=isinside_w51n_b3_woff)

multiplicity_w51e_wff_b3 = multiplicity(hdrhr_w51e_b3,cat_w51e,xysource_w51e,isb3=False,
                                 beam = beamb6_w51e_almaimf,
                                 use_criteria='both',
                                 label=isinside_w51e_b3_wff)
multiplicity_w51n_wff_b3 = multiplicity(hdrhr_w51n_b3,cat_w51n,xysource_w51n,isb3=False,
                                 beam = beamb6_w51n_almaimf,
                                 use_criteria='both',
                                label=isinside_w51n_b3_wff)

ysoflux_upperr_b6_w51e = np.array(catalog_w51e_yso['flux_upperr_b6']) 
ysoflux_upperr_b3_w51e = np.array(catalog_w51e_yso['flux_upperr_b3'])
ysoflux_lowerr_b6_w51e = np.array(catalog_w51e_yso['flux_lowerr_b6'])
ysoflux_lowerr_b3_w51e = np.array(catalog_w51e_yso['flux_lowerr_b3']) 

ysoflux_upperr_b6_w51n = np.array(catalog_w51n_yso['flux_upperr_b6']) 
ysoflux_upperr_b3_w51n = np.array(catalog_w51n_yso['flux_upperr_b3']) 
ysoflux_lowerr_b6_w51n = np.array(catalog_w51n_yso['flux_lowerr_b6']) 
ysoflux_lowerr_b3_w51n = np.array(catalog_w51n_yso['flux_lowerr_b3']) 

ysoflux_b6_w51e = catalog_w51e_yso['flux_b6']
ysoflux_b3_w51e = catalog_w51e_yso['flux_b3']
ysoflux_b6_w51n = catalog_w51n_yso['flux_b6']
ysoflux_b3_w51n = catalog_w51n_yso['flux_b3']

ysoflux_b6_w51e = catalog_w51e_yso['flux_b6']
ysoflux_b3_w51e = catalog_w51e_yso['flux_b3']
ysoflux_b6_w51n = catalog_w51n_yso['flux_b6']
ysoflux_b3_w51n = catalog_w51n_yso['flux_b3']

ysoflux_b6_w51e = np.array([np.nan if flux<0 else flux for flux in ysoflux_b6_w51e])
ysoflux_b3_w51e = np.array([np.nan if flux<0 else flux for flux in ysoflux_b3_w51e])
ysoflux_b6_w51n = np.array([np.nan if flux<0 else flux for flux in ysoflux_b6_w51n])
ysoflux_b3_w51n = np.array([np.nan if flux<0 else flux for flux in ysoflux_b3_w51n])


coreindarr_w51e_woff = multiplicity_w51e_woff.coreindarr
coreindarr_w51n_woff = multiplicity_w51n_woff.coreindarr
coreindarr_w51e_woff_b3 = multiplicity_w51e_woff_b3.coreindarr
coreindarr_w51n_woff_b3 = multiplicity_w51n_woff_b3.coreindarr
coreindarr_w51e_wff = multiplicity_w51e_wff.coreindarr
coreindarr_w51n_wff = multiplicity_w51n_wff.coreindarr
coreindarr_w51e_wff_b3 = multiplicity_w51e_wff_b3.coreindarr
coreindarr_w51n_wff_b3 = multiplicity_w51n_wff_b3.coreindarr

ysoindarr_w51e_woff = multiplicity_w51e_woff.ysoindarr
ysoindarr_w51n_woff = multiplicity_w51n_woff.ysoindarr
ysoindarr_w51e_woff_b3 = multiplicity_w51e_woff_b3.ysoindarr
ysoindarr_w51n_woff_b3 = multiplicity_w51n_woff_b3.ysoindarr
ysoindarr_w51e_wff = multiplicity_w51e_wff.ysoindarr
ysoindarr_w51n_wff = multiplicity_w51n_wff.ysoindarr
ysoindarr_w51e_wff_b3 = multiplicity_w51e_wff_b3.ysoindarr
ysoindarr_w51n_wff_b3 = multiplicity_w51n_wff_b3.ysoindarr

coreflux_w51e_b6_woff = flux_w51e_b6[coreindarr_w51e_woff]
coreflux_w51e_b3_woff = flux_w51e_b3[coreindarr_w51e_woff_b3]
coreflux_w51n_b6_woff = flux_w51n_b6[coreindarr_w51n_woff]
coreflux_w51n_b3_woff = flux_w51n_b3[coreindarr_w51n_woff_b3]

coreflux_w51e_b6_wff = flux_w51e_b6[coreindarr_w51e_wff]
coreflux_w51e_b3_wff = flux_w51e_b3[coreindarr_w51e_wff_b3]
coreflux_w51n_b6_wff = flux_w51n_b6[coreindarr_w51n_wff]
coreflux_w51n_b3_wff = flux_w51n_b3[coreindarr_w51n_wff_b3]

corefluxerr_w51e_b6_woff = fluxerr_w51e_b6[coreindarr_w51e_woff]
corefluxerr_w51e_b3_woff = fluxerr_w51e_b3[coreindarr_w51e_woff_b3]

corefluxerr_w51n_b6_woff = fluxerr_w51n_b6[coreindarr_w51n_woff]
corefluxerr_w51n_b3_woff = fluxerr_w51n_b3[coreindarr_w51n_woff_b3]

corefluxerr_w51e_b6_wff = fluxerr_w51e_b6[coreindarr_w51e_woff]
corefluxerr_w51e_b3_wff = fluxerr_w51e_b3[coreindarr_w51e_woff_b3]

corefluxerr_w51n_b6_wff = fluxerr_w51n_b6[coreindarr_w51n_wff]
corefluxerr_w51n_b3_wff = fluxerr_w51n_b3[coreindarr_w51n_wff_b3]

coremass_w51e_b6_woff = coremass_w51e[coreindarr_w51e_woff]
coremass_w51e_b3_woff = coremass_w51e[coreindarr_w51e_woff_b3]

coremass_w51n_b6_woff = coremass_w51n[coreindarr_w51n_woff]
coremass_w51n_b3_woff = coremass_w51n[coreindarr_w51n_woff_b3]

coremass_w51e_b6_wff = coremass_w51e[coreindarr_w51e_wff]
coremass_w51e_b3_wff = coremass_w51e[coreindarr_w51e_wff_b3]

coremasserr_w51n_b6_wff = coremasserr_w51n[coreindarr_w51n_wff]
coremasserr_w51n_b3_wff = coremasserr_w51n[coreindarr_w51n_wff_b3]

coremasserr_w51e_b6_woff = coremasserr_w51e[coreindarr_w51e_woff]
coremasserr_w51e_b3_woff = coremasserr_w51e[coreindarr_w51e_woff_b3]

coremasserr_w51n_b6_woff = coremasserr_w51n[coreindarr_w51n_woff]
coremasserr_w51n_b3_woff = coremasserr_w51n[coreindarr_w51n_woff_b3]

coremasserr_w51e_b6_wff = coremasserr_w51e[coreindarr_w51e_wff]
coremasserr_w51e_b3_wff = coremasserr_w51e[coreindarr_w51e_wff_b3]

coremasserr_w51n_b6_wff = coremasserr_w51n[coreindarr_w51n_wff]
coremasserr_w51n_b3_wff = coremasserr_w51n[coreindarr_w51n_wff_b3]

coretemp_w51e_b6_woff = coretemp_w51e[coreindarr_w51e_woff]
coretemp_w51e_b3_woff = coretemp_w51e[coreindarr_w51e_woff_b3]
coretemp_w51n_b6_woff = coretemp_w51n[coreindarr_w51n_woff]
coretemp_w51n_b3_woff = coretemp_w51n[coreindarr_w51n_woff_b3]

coretemp_w51e_b6_wff = coretemp_w51e[coreindarr_w51e_wff]
coretemp_w51e_b3_wff = coretemp_w51e[coreindarr_w51e_wff_b3]
coretemp_w51n_b6_wff = coretemp_w51n[coreindarr_w51n_wff]
coretemp_w51n_b3_wff = coretemp_w51n[coreindarr_w51n_wff_b3]




ysoflux_w51e_b6_woff = [ysoflux_b6_w51e[ind] for ind in ysoindarr_w51e_woff]
ysoflux_w51e_b3_woff = [ysoflux_b3_w51e[ind] for ind in ysoindarr_w51e_woff_b3] 
ysoflux_w51e_b6_woff = [ysoflux_b6_w51e[ind] for ind in ysoindarr_w51e_woff]
ysoflux_w51e_b3_woff = [ysoflux_b3_w51e[ind] for ind in ysoindarr_w51e_woff_b3] 

ysoflux_w51n_b6_woff = [ysoflux_b6_w51n[ind] for ind in ysoindarr_w51n_woff]
ysoflux_w51n_b3_woff = [ysoflux_b3_w51n[ind] for ind in ysoindarr_w51n_woff_b3] 
ysoflux_w51n_b6_woff = [ysoflux_b6_w51n[ind] for ind in ysoindarr_w51n_woff]
ysoflux_w51n_b3_woff = [ysoflux_b3_w51n[ind] for ind in ysoindarr_w51n_woff_b3] 

ysoflux_upperr_w51e_b6_woff = [ysoflux_upperr_b6_w51e[ind] for ind in ysoindarr_w51e_woff]
ysoflux_upperr_w51e_b3_woff = [ysoflux_upperr_b3_w51e[ind] for ind in ysoindarr_w51e_woff_b3] 
ysoflux_lowerr_w51e_b6_woff = [ysoflux_lowerr_b6_w51e[ind] for ind in ysoindarr_w51e_woff]
ysoflux_lowerr_w51e_b3_woff = [ysoflux_lowerr_b3_w51e[ind] for ind in ysoindarr_w51e_woff_b3] 

ysoflux_upperr_w51n_b6_woff = [ysoflux_upperr_b6_w51n[ind] for ind in ysoindarr_w51n_woff]
ysoflux_upperr_w51n_b3_woff = [ysoflux_upperr_b3_w51n[ind] for ind in ysoindarr_w51n_woff_b3] 
ysoflux_lowerr_w51n_b6_woff = [ysoflux_lowerr_b6_w51n[ind] for ind in ysoindarr_w51n_woff]
ysoflux_lowerr_w51n_b3_woff = [ysoflux_lowerr_b3_w51n[ind] for ind in ysoindarr_w51n_woff_b3] 

ysoflux_w51e_b6_wff = [ysoflux_b6_w51e[ind] for ind in ysoindarr_w51e_wff]
ysoflux_w51e_b3_wff = [ysoflux_b3_w51e[ind] for ind in ysoindarr_w51e_wff_b3] 
ysoflux_w51e_b6_wff = [ysoflux_b6_w51e[ind] for ind in ysoindarr_w51e_wff]
ysoflux_w51e_b3_wff = [ysoflux_b3_w51e[ind] for ind in ysoindarr_w51e_wff_b3] 

ysoflux_w51n_b6_wff = [ysoflux_b6_w51n[ind] for ind in ysoindarr_w51n_wff]
ysoflux_w51n_b3_wff = [ysoflux_b3_w51n[ind] for ind in ysoindarr_w51n_wff_b3] 
ysoflux_w51n_b6_wff = [ysoflux_b6_w51n[ind] for ind in ysoindarr_w51n_wff]
ysoflux_w51n_b3_wff = [ysoflux_b3_w51n[ind] for ind in ysoindarr_w51n_wff_b3] 

ysoflux_upperr_w51e_b6_wff = [ysoflux_upperr_b6_w51e[ind] for ind in ysoindarr_w51e_wff]
ysoflux_upperr_w51e_b3_wff = [ysoflux_upperr_b3_w51e[ind] for ind in ysoindarr_w51e_wff_b3] 
ysoflux_lowerr_w51e_b6_wff = [ysoflux_lowerr_b6_w51e[ind] for ind in ysoindarr_w51e_wff]
ysoflux_lowerr_w51e_b3_wff = [ysoflux_lowerr_b3_w51e[ind] for ind in ysoindarr_w51e_wff_b3] 

ysoflux_upperr_w51n_b6_wff = [ysoflux_upperr_b6_w51n[ind] for ind in ysoindarr_w51n_wff]
ysoflux_upperr_w51n_b3_wff = [ysoflux_upperr_b3_w51n[ind] for ind in ysoindarr_w51n_wff_b3] 
ysoflux_lowerr_w51n_b6_wff = [ysoflux_lowerr_b6_w51n[ind] for ind in ysoindarr_w51n_wff]
ysoflux_lowerr_w51n_b3_wff = [ysoflux_lowerr_b3_w51n[ind] for ind in ysoindarr_w51n_wff_b3] 

cores with/without YSOs  21 6
YSOs associated with cores 53  out of  137
cores with/without YSOs  38 53
YSOs associated with cores 57  out of  74
cores with/without YSOs  16 2
YSOs associated with cores 45  out of  137
cores with/without YSOs  28 29
YSOs associated with cores 41  out of  74
cores with/without YSOs  4 3
YSOs associated with cores 7  out of  137
cores with/without YSOs  5 11
YSOs associated with cores 11  out of  74
cores with/without YSOs  13 1
YSOs associated with cores 39  out of  137
cores with/without YSOs  15 7
YSOs associated with cores 32  out of  74
cores with/without YSOs  10 0
YSOs associated with cores 33  out of  137
cores with/without YSOs  12 2
YSOs associated with cores 23  out of  74
cores with/without YSOs  3 1
YSOs associated with cores 6  out of  137
cores with/without YSOs  3 5
YSOs associated with cores 9  out of  74


In [24]:
import os.path
from scipy import stats

def get_error_eps(fluxdir, fluxfits, objid, sym_mean, sym_sigma, n_samples=2000, ):
    
    #fig = plt.figure(figsize=(24,8))
    #ax1 = fig.add_axes([0.1,0.1,0.24,0.8])
    #ax2 = fig.add_axes([0.4,0.1,0.24,0.8])
    #ax3 = fig.add_axes([0.7,0.1,0.24,0.8])
    arr = []
    for i in range(len(objid)):
        if os.path.isfile(fluxdir+'_%03d_flux.dat'%objid[i]):
            flux = np.loadtxt(fluxdir+'_%03d_flux.dat'%objid[i])
            kernel = stats.gaussian_kde(np.log10(flux))
            xarr = np.linspace(np.nanmin(np.log10(flux)), np.nanmax(np.log10(flux)),n_samples*10)
            pdf = kernel(xarr)
            samples_flux = 10**np.random.choice(xarr, size=n_samples, p = pdf/np.sum(pdf))
            #bin = np.logspace(np.min(np.log10(flux)), np.max(np.log10(flux)), 100)
            #ax1.hist(flux, bin=bin)
            print(i, objid[i], np.median(flux))

        else:
            fluxtab = Table.read(fluxfits)
            source_flux = fluxtab['flux'][objid[i]]
            source_fluxerr = fluxtab['flux_upperr'][objid[i]]
            samples_flux = 10**np.random.normal(np.log10(source_flux), np.log10(source_fluxerr/source_flux), n_samples)
            if not np.isfinite(source_flux):
                continue
        
        if len(objid) > 1:
            arr.append(samples_flux)
    if len(objid) > 1:
        arr_stack = np.vstack(arr)
        samples_flux = np.sum(arr_stack, axis=0)
    sym_gauss_pdf = np.random.normal(sym_mean, sym_sigma, size=n_samples)
    eps = samples_flux / sym_gauss_pdf
    


    
    
                            

    return eps

def run_get_error_eps(fluxdir, fluxfits, objid, core_flux, core_fluxerr,n_samples=2000, savelabel='w51e_b3'):
    fluxtab = Table.read(fluxfits)

    eps_low_arr = []
    eps_med_arr = []
    eps_upp_arr = []
    
    for i in range(len(core_flux)):
        if len(objid[i])>0:
            if any([os.path.isfile(fluxdir+'_%03d_flux.dat'%ids) for ids in objid[i]]):

                eps = get_error_eps(fluxdir, fluxfits, objid[i], core_flux[i], core_fluxerr[i], n_samples=n_samples)
                eps_low, eps_med, eps_upp = np.percentile(eps, [16, 50, 84])

                print(i, eps_low, eps_med, eps_upp, np.sum(fluxtab['flux'][objid[i]])/core_flux[i])
                eps_low_arr.append(eps_low)
                eps_med_arr.append(eps_med)
                eps_upp_arr.append(eps_upp)
        
    np.savetxt(savelabel+'_eps_low.dat', eps_low_arr, newline='\n', delimiter=' ')
    np.savetxt(savelabel+'_eps_med.dat', eps_med_arr, newline='\n', delimiter=' ')
    np.savetxt(savelabel+'_eps_upp.dat', eps_upp_arr, newline='\n', delimiter=' ')

    
    return eps_low_arr, eps_med_arr, eps_upp_arr

eps_low, eps_med, eps_upp = run_get_error_eps('/home/t.yoo/w51/catalogue/photometry/pdf/w51e_b3', '/home/t.yoo/w51/catalogue/photometry/w51e_b3_flux_size.fits',
                                             ysoindarr_w51e_woff_b3, coreflux_w51e_b3_woff, corefluxerr_w51e_b3_woff, savelabel='/home/t.yoo/w51/catalogue/comparison_almaimf/w51e_b3')


eps_low, eps_med, eps_upp = run_get_error_eps('/home/t.yoo/w51/catalogue/photometry/pdf/w51n_b3', '/home/t.yoo/w51/catalogue/photometry/w51n_b3_flux_size.fits',
                                             ysoindarr_w51n_woff_b3, coreflux_w51n_b3_woff, corefluxerr_w51n_b3_woff, savelabel='/home/t.yoo/w51/catalogue/comparison_almaimf/w51n_b3')

eps_low, eps_med, eps_upp = run_get_error_eps('/home/t.yoo/w51/catalogue/photometry/pdf/w51e_b6_conv', '/home/t.yoo/w51/catalogue/photometry/w51e_b6_conv_flux_size.fits',
                                             ysoindarr_w51e_woff, coreflux_w51e_b6_woff, corefluxerr_w51e_b6_woff, savelabel='/home/t.yoo/w51/catalogue/comparison_almaimf/w51e_b6_conv')


eps_low, eps_med, eps_upp = run_get_error_eps('/home/t.yoo/w51/catalogue/photometry/pdf/w51n_b6_conv', '/home/t.yoo/w51/catalogue/photometry/w51n_b6_conv_flux_size.fits',
                                             ysoindarr_w51n_woff, coreflux_w51n_b6_woff, corefluxerr_w51n_b6_woff, savelabel='/home/t.yoo/w51/catalogue/comparison_almaimf/w51n_b6_conv')
                  

0 40 0.02850327607190835
0 0.0958563014663544 0.12035447378871336 0.14865244503697708 0.11979550535635397
0 29 0.0056859430793064275
1 30 0.001016086799295505
2 31 0.0028769499326077594
3 32 0.0031254551819847078
4 33 0.02106379562609516
5 34 0.0011790514748138774
1 0.23118962683118008 0.27679447097573884 0.33143562634630563 0.2593624068589582
0 46 0.001547266701387732
1 48 0.009725674675350859
2 50 0.003178145636727696
3 51 0.0012420874567077161
4 54 0.0008236594591049281
5 56 0.0012056863817976405
2 0.28567193464103713 0.320281953513164 0.3677656661813898 0.3118248019261459
0 52 0.0028740453080876812
1 55 0.0007609251076876397
2 58 0.0009571203292070883
3 59 0.0035586092224018074
4 60 0.0017044486027193578
5 61 0.0012068291074477206
3 0.3192893547834496 0.361252502522902 0.4135301079021868 0.3550952811867738
0 37 0.0025144177871911045
4 0.2784667470672073 0.30849073270735355 0.34265401627986913 0.30747633820225023
0 1 0.0021551809201017557
1 2 0.0006577924781379246
2 3 0.000283101822